In [ ]:
%load_ext autoreload
%autoreload 2

import pdb
import pickle, os
import numpy as np
import cvxpy as cp
import concurrent.futures


import matplotlib.pyplot as plt
%matplotlib inline

from free_flyer import FreeFlyer
from utils import *

In [2]:
n = 2; m = 2

N = 6 # horizon
dh = 0.75
Ak = np.eye(2*n)
Ak[:n,n:] = dh*np.eye(n)
Bk = np.zeros((2*n,m))
Bk[:n,:] = 0.5*dh**2 * np.eye(n)
Bk[n:,:] = dh*np.eye(n)

Q = np.diag([2,2,1,1.])
R = 1000.*np.eye(m)

mass_ff_min = 15.36
mass_ff_max = 18.08
mass_ff = 0.5*(mass_ff_min+mass_ff_max)
thrust_max = 2*1.  # max thrust [N] from two thrusters
umin = -thrust_max/mass_ff
umax = thrust_max/mass_ff
velmin = -0.2
velmax = 0.2
posmin = np.zeros(n)

ft2m = 0.3048
posmax = ft2m*np.array([12.,9.])
max_box_size = 0.75
min_box_size = 0.25
box_buffer = 0.025
border_size = 0.05

n_obs = 8

prob_params = [N, Ak, Bk, Q, R, n_obs , \
    posmin, posmax, velmin, velmax, umin, umax]

In [3]:
#setup filenames
relative_path = os.getcwd()
dataset_name = 'default_index'

if not os.path.isdir(os.path.join(relative_path, 'data', dataset_name)):
    os.mkdir(os.path.join(relative_path+'/data/'+dataset_name))

if not os.path.isdir(os.path.join(relative_path, 'config')):
    os.mkdir(os.path.join(relative_path, 'config'))

train_fn = os.path.join(relative_path, 'data', dataset_name, 'train.p')
test_fn = os.path.join(relative_path, 'data', dataset_name, 'test.p')

#load default parameter values
n = 2; m = 2

#define all possible params that can be varied
all_params = ['N', 'Ak', 'Bk', 'Q', 'R', 'n_obs', \
    'posmin', 'posmax', 'velmin', 'velmax', \
    'umin', 'umax', \
    'x0', 'xg', 'obstacles']

## TODO(pculbertson): implement custom param variation.
#plan: split all_params into static & sampled params
#in Problem object, keep param dict with keys == all_params
#on constructing problem, make every param key map to either static value
#or a cvxpy.Parameter object.

### VARY ANY DESIRED PARAM VALUES HERE ###

param_dict = {'N':N, 'Ak':Ak, 'Bk':Bk, 'Q':Q, 'R':R, 'n_obs':n_obs, \
    'posmin':posmin, 'posmax':posmax, 'velmin':velmin, 'velmax':velmax, \
    'umin':umin, 'umax':umax}

#specify which parameters to sample, & their distributions
sampled_params = ['x0', 'xg', 'obstacles']
sample_dists = {'x0': lambda: np.hstack((posmin + (posmax-posmin)*np.random.rand(2), \
               velmin + (velmax-velmin)*np.random.rand(2))) ,\
               'xg': lambda: np.hstack((0.9*posmax, np.zeros(n))), \
               'obstacles': lambda: random_obs(n_obs, posmin, posmax, border_size, min_box_size, max_box_size)}

#specify dataset sizes
num_train = 90; num_test = 10
num_probs = num_train + num_test

#write out config
config_fn = os.path.join(relative_path, 'config', dataset_name+'.p')

config = [dataset_name, prob_params, sampled_params]
outfile = open(config_fn,"wb")
pickle.dump(config,outfile); outfile.close()

In [4]:
prob = FreeFlyer()

#create numpy containers for data: (params, x, u, y, J*, solve_time)
params = {}
if 'x0' in sampled_params:
    params['x0'] = np.zeros((num_probs,2*n))
if 'xg' in sampled_params:
    params['xg'] = np.zeros((num_probs,2*n))
if 'obstacles' in sampled_params:
    params['obstacles'] = np.zeros((num_probs, 4, n_obs))

X = np.zeros((num_probs, 2*n, N));
U = np.zeros((num_probs, m, N-1))
Y = np.zeros((num_probs, 4*n_obs, N-1)).astype(int)

costs = np.zeros(num_probs)
solve_times = np.zeros(num_probs)

In [5]:
total_count = 0
ii = 0
ii_obs = 0
ii_toggle = 0
obs_new_ct = 5
toggle_ct = 10
toggle_obstacles = False

#big for loop, sampling problem data, solving MICP, & storing
while ii < num_probs:
    total_count += 1
    if total_count % 100 == 0:
        print('Done with problem {}'.format(ii))
#         file = open('testfile_long.txt','w')
#         file.write('total_count: {}\n'.format(total_count))
#         file.write('Problems solved: {}'.format(ii))
#         file.close()

    p_dict = {}
    if 'x0' in sampled_params or 'obstacles' in sampled_params:
        if toggle_obstacles:#两种初始化点和obs的方法#
            x0 = sample_dists['x0']()
            obstacles = find_obs(x0, n_obs, posmin, posmax,\
                border_size, box_buffer, min_box_size, max_box_size, \
                ignore_intersection=False)#给x0再找obs，所以不忽略obs可能相交#
            if len(obstacles) is not n_obs:#如果找完obs后，obs数量还不够，则忽略约束再生成obs#
                obstacles = find_obs(x0, n_obs, posmin, posmax,\
                    border_size, box_buffer, min_box_size, max_box_size, \
                    ignore_intersection=True)

            if ii_toggle % obs_new_ct == 0:
                toggle_obstacles = False
                ii_obs = 0
                ii_toggle = 0
        else:#给定obs后再找obs外的x0#
            if ii_obs % obs_new_ct == 0:
                obstacles = \
                    random_obs(n_obs, posmin, posmax, border_size, box_buffer, min_box_size, max_box_size)
            x0 = findIC(obstacles, posmin, posmax, velmin, velmax)

            if ii_toggle % toggle_ct == 0:
                toggle_obstacles = True
                ii_obs = 0
                ii_toggle = 0

        if len(obstacles) is not n_obs:
            continue

        params['obstacles'][ii,:] = np.reshape(np.concatenate(obstacles, axis=0), (n_obs,4)).T
        params['x0'][ii,:] = x0

        p_dict['x0'] = params['x0'][ii,:]
        p_dict['obstacles'] = params['obstacles'][ii,:]

    if 'xg' in sampled_params:
        params['xg'][ii,:] = sample_dists['xg']()
        p_dict['xg'] = params['xg'][ii,:]

#     dump_file = open('dump_data.p','wb')
#     dump_data = [p_dict['x0'], p_dict['xg'], p_dict['obstacles']]
#     pickle.dump(dump_data,dump_file); dump_file.close()

    prob_success = False
    try:
#         with time_limit(20):
        prob_success, cost, solve_time, optvals = prob.solve_micp(p_dict, solver=cp.GUROBI)
    except (KeyboardInterrupt, SystemExit):
        raise
    except:
        print('solver failed at '.format(ii))
    #     except (TimeoutException):
#         print("Timed out at {}!".format(ii))

    if prob_success:
        costs[ii] = cost; solve_times[ii] = solve_time
        X[ii,:,:], U[ii,:,:], Y[ii,:,:] = optvals
        ii += 1
        ii_obs += 1
        ii_toggle += 1

Academic license - for non-commercial use only - expires 2022-12-22
Using license file C:\Users\msi\gurobi.lic
Done with problem 87


In [6]:
arr = np.arange(num_probs)
np.random.shuffle(arr)

X = X[arr]
U = U[arr]
Y = Y[arr]

if 'x0' in sampled_params:
    params['x0'] = params['x0'][arr]
if 'xg' in sampled_params:
    params['xg'] = params['xg'][arr]
if 'obstacles' in sampled_params:
    params['obstacles'] = params['obstacles'][arr]

costs = costs[arr]
solve_times = solve_times[arr]

In [7]:
#post-processing + write out
train_params = {}; test_params = {}
if 'x0' in sampled_params:
    train_params['x0'] = params['x0'][:num_train,:]
    test_params['x0'] = params['x0'][num_train:,:]
if 'obstacles' in sampled_params:
    train_params['obstacles'] = params['obstacles'][:num_train,:]
    test_params['obstacles'] = params['obstacles'][num_train:,:]
if 'xg' in sampled_params:
    train_params['xg'] = params['xg'][:num_train,:]
    test_params['xg'] = params['xg'][num_train:,:]

train_data = [train_params] #train_data: {x0, obstacles, xg}, X,U,Y,COST,SOLVE_TIME
train_data += [X[:num_train,:,:], U[:num_train,:,:], Y[:num_train,:,:]]
train_data += [costs[:num_train], solve_times[:num_train]]

test_data = [test_params]
test_data += [X[num_train:,:,:], U[num_train:,:,:], Y[num_train:,:,:]]
test_data += [costs[num_train:], solve_times[num_train:]]

train_file = open(train_fn,'wb')
pickle.dump(train_data,train_file); train_file.close()

test_file = open(test_fn, 'wb')
pickle.dump(test_data,test_file); test_file.close()